In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel

# 예시 데이터
samples = [
    ("Are you sure you want to cancel?", "contains_phrase(screen1, confirmshaming_phrase)"),
    ("Limited time offer!", "contains_phrase(screen2, fake_urgency_phrase)"),
    ("Only 3 left in stock!", "contains_phrase(screen3, fake_scarcity_phrase)"),
]

# 라벨 인코딩
texts, labels = zip(*samples)
label_encoder = LabelEncoder()
label_ids = label_encoder.fit_transform(labels)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Dataset 클래스
class PredicateDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(list(texts), padding=True, truncation=True, return_tensors="pt")
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'token_type_ids': self.encodings['token_type_ids'][idx],
            'labels': self.labels[idx]
        }

# Train/Val split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, label_ids, test_size=0.2, random_state=42)
train_dataset = PredicateDataset(train_texts, train_labels)
val_dataset = PredicateDataset(val_texts, val_labels)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

# 모델 정의
class PredicateClassifier(nn.Module):
    def __init__(self, num_classes):
        super(PredicateClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)
        return logits

# 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PredicateClassifier(num_classes=len(label_encoder.classes_)).to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 훈련 루프
for epoch in range(3):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"✅ Epoch {epoch+1} completed")

# 예측 함수
def predict_predicate(text):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        logits = model(**inputs)
        predicted = torch.argmax(logits, dim=1).cpu().numpy()[0]
        return label_encoder.inverse_transform([predicted])[0]

# 예측 예시
test_text = "Only 3 left in stock!"
print("🧠 예측된 Predicate:", predict_predicate(test_text))

✅ Epoch 1 completed
✅ Epoch 2 completed
✅ Epoch 3 completed
🧠 예측된 Predicate: contains_phrase(screen3, fake_scarcity_phrase)
